In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2
import random
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
# import shap
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import copy

data = pd.read_csv('df_data (1).csv', sep=";")
data["ГБ новая"] = data["ГБ новая"].replace({'да': 1, 'нет': 0})
data_not_import = data[["startAnswering", "id", "комментарий"]]
data = data.drop(columns=["startAnswering", "id", "комментарий"], axis=1)
categorical_columns = [
                       "сыр, шоко, цитрус", "Физическая ативность", ]
binary_columns = ['Жажда', 'кофеин', 'Зевания', 'ГБ новая', 'Алкоголь'
                  , "боль в шее", "вегетатика", "подташнивает",
                   "мочеиспускание", "Напряжение глаз", "Воды достаточно?", "Были резкие запахи?",
                    "Пропуск приема пищи", "Чувствит кожи головы",
                   "Хотелось шоколада"]
quantitative_columns = ["Тревога", "Депрессия", "Сонливость", "Чувство голода",
                        "Работосособность", "Чувство усталости", "Раздражительность"

                        ]
time_data =['time']
date_data = ["date"]
time_data =['time']
date_data = ["date"]
longest_string_index = data[['time', "Начало боли", "Продолжительность сна", "Окончание боли"]].apply(lambda x: ''.join(x.astype(str)), axis=1).str.len().idxmax()
longest_string = data.loc[longest_string_index, ['time', "Начало боли", "Продолжительность сна", "Окончание боли"]]
data = data.drop(columns=["Боль сейчас"], axis=1)
data = data.drop(data.iloc[:, 27:58],axis = 1)
data = data.drop(columns=["Ожидаете мигрень"], axis=1)
mode_values = data.mode().iloc[0]

# Fill NaN values with mode values in all columns
data = data.fillna(mode_values)
data["ГБ новая"] = data["ГБ новая"].astype(int)
target = data["ГБ новая"]
y = data["ГБ новая"].values
data = data.drop(columns = ["ГБ новая"], axis=1)
binary_columns = ['Жажда', 'кофеин', 'Зевания',  'Алкоголь'
                  , "боль в шее", "вегетатика", "подташнивает",
                   "мочеиспускание", "Напряжение глаз", "Воды достаточно?", "Были резкие запахи?",
                    "Пропуск приема пищи", "Чувствит кожи головы",
                   "Хотелось шоколада"]

for column in binary_columns:
    data[column] = data[column].replace({'да': 1, 'нет': 0})

for column in time_data:
    data[column] = pd.to_datetime(data[column], format='%H:%M')

for column in time_data:
    time_components = [f'{column}_hour', f'{column}_minute']

    data[time_components] = data[column].apply(lambda x: pd.Series([x.hour, x.minute]))
    data_encoded = pd.get_dummies(data, columns=time_components)


data = pd.get_dummies(data, columns=categorical_columns+quantitative_columns+binary_columns)

data["time"] = pd.to_datetime(data["time"]).dt.time
data["datetime"] = pd.to_datetime(data["date"] + " " + data["time"].astype(str))

# Set the datetime column as the index
# data.set_index("datetime", inplace=True)

# Drop unnecessary columns
data.drop(["date", "time", "datetime"], axis=1, inplace=True)

<ipython-input-3-7ab852ae96dd>:34: DtypeWarning: Columns (11,14,20,23,31,33,41,46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('df_data (1).csv', sep=";")


In [4]:
X = data.iloc[:, 2:].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=True)

1 Run for params

In [2]:
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time
# Define the hyperparameters for each model
params_lr = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'penalty': ['l1', 'l2']}
params_rf = {'n_estimators': [100, 200, 300, 400, 500], 'max_depth': [5, 10, 15, 20, None]}
params_svm = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'gamma': [0.001, 0.01, 0.1, 1]}
params_gb = {'n_estimators': [100, 200, 300], 'learning_rate': [0.1, 0.05, 0.01], 'max_depth': [3, 4, 5]}
params_knn = {'n_neighbors': list(range(1, 31)), 'weights': ['uniform', 'distance']}
params_lgbm = {'n_estimators': [100, 200, 300], 'learning_rate': [0.1, 0.05, 0.01], 'max_depth': [3, 4, 5]}

# Define models
models = [
    ('LR', LogisticRegression(), params_lr),
    ('RF', RandomForestClassifier(), params_rf),
    ('SVM', SVC(probability=True), params_svm),
    ('GB', GradientBoostingClassifier(), params_gb),
    ('KNN', KNeighborsClassifier(), params_knn),
    ('LGBM', LGBMClassifier(), params_lgbm)
]

for name, model, params in models:
    gs = GridSearchCV(model, params, cv=5, scoring='roc_auc', n_jobs=-1)
    gs.fit(X_train, y_train)
    print(f"Best parameters for {name}: {gs.best_params_}")
    print(f"Best score for {name}: {gs.best_score_}")

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
35 fits failed out of a total of 70.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solve

Best parameters for LR: {'C': 0.01, 'penalty': 'l2'}
Best score for LR: 0.7272352204039667


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters for RF: {'max_depth': 10, 'n_estimators': 400}
Best score for RF: 0.732485074109838


2 run for params

In [ ]:
# More extensive hyperparameters for each model
params_lr = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
             'penalty': ['l1', 'l2', 'elasticnet', 'none'],
             'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

params_rf = {'n_estimators': [100, 200, 300, 400, 500],
             'max_depth': [5, 10, 15, 20, None],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
             'bootstrap': [True, False]}

params_svm = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

params_gb = {'n_estimators': [100, 200, 300, 400, 500],
             'learning_rate': [0.1, 0.05, 0.01, 0.005, 0.001],
             'max_depth': [3, 4, 5, 6, 7, 8],
             'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
             'min_samples_split': [2, 5, 10]}

params_knn = {'n_neighbors': list(range(1, 31)),
              'weights': ['uniform', 'distance'],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1, 2]}

params_lgbm = {'n_estimators': [100, 200, 300, 400, 500],
               'learning_rate': [0.1, 0.05, 0.01, 0.005, 0.001],
               'max_depth': [-1, 3, 4, 5, 6, 7, 8],
               'num_leaves': [7, 14, 31, 62, 125, 250, 500, 1000],
               'min_child_samples': [5, 10, 20, 30]}

# Define models
models = [
    ('LR', LogisticRegression(max_iter=5000), params_lr),
    ('RF', RandomForestClassifier(), params_rf),
    ('SVM', SVC(probability=True), params_svm),
    ('GB', GradientBoostingClassifier(), params_gb),
    ('KNN', KNeighborsClassifier(), params_knn),
    ('LGBM', LGBMClassifier(), params_lgbm)
]

for name, model, params in models:
    gs = GridSearchCV(model, params, cv=5, scoring='roc_auc', n_jobs=-1, verbose=2)
    gs.fit(X_train, y_train)
    print(f"Best parameters for {name}: {gs.best_params_}")
    print(f"Best score for {name}: {gs.best_score_}")
